<div class="alert">
<div style="direction:ltr;text-align:center;font-family: B Tahoma; font-size:24pt"> Practical Deep Learning Course for Computer Vision
</div>
</div>

<div class="alert">
<div style="direction:ltr;text-align:left;font-family:Tahoma; font-size:16pt"> Transfer Learning
</div>
</div>

<img src="./images/catdog.jpg" alt="Cat Vs. Dog">
<a href="https://apopka-1x1yusplq.stackpathdns.com/wp-content/uploads/2017/12/gfhdgfgfgdfgdf.jpg">Reference</a>

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Download the dataset.
</div>

In [ ]:
!wget http://files.fast.ai/data/dogscats.zip

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Unzip the dataset.
</div>

In [ ]:
!unzip -qq dogscats.zip

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Import the required libraries.<br>
</div>

In [ ]:
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
import keras
import matplotlib.pyplot as plt

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Setting the directory for the dataset:<br>
</div>

In [ ]:
PATH_TRAIN = "train"
PATH_TEST = "validation"

BUFFER_SIZE = 128
BATCH_SIZE = 128
EPOCHS = 90

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Defining dataset generator:
</div>

In [ ]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return parts[-2] == CLASS_NAMES


def load(image_file):
    input_image = tf.io.read_file(image_file)
    input_image = tf.image.decode_jpeg(input_image)
    input_image = tf.cast(input_image, tf.float32)
    label = get_label(image_file)

    return input_image, label


def resize(input_image, height, width):
    input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    return input_image


# normalizing the images to [0, 1]
def normalize(input_image):
    return input_image / 255.


@tf.function()
def aug(input_image):
    input_image = resize(input_image, 256, 256)
    cropped_image = tf.image.random_crop(input_image, size=[IMG_HEIGHT, IMG_WIDTH, CHANNELS])

    if tf.random.uniform(()) > 0.5:
        # random mirroring
        cropped_image = tf.image.flip_left_right(cropped_image)

    return cropped_image


def load_image_train(image_file):
    input_image, label = load(image_file)
    input_image = aug(input_image)
    input_image = normalize(input_image)

    return input_image, label


def load_image_test(image_file):
    input_image, label = load(image_file)
    input_image = resize(input_image, IMG_HEIGHT, IMG_WIDTH)
    input_image = normalize(input_image)

    return input_image, label


In [ ]:
# input pipeline
CLASS_NAMES = np.array(os.listdir(PATH_TRAIN))

train_dataset = tf.data.Dataset.list_files(PATH_TRAIN+'/*/*.jpg')
train_dataset = train_dataset.map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.list_files(PATH_TEST+'*/*.jpg')
val_dataset = val_dataset.map(load_image_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE)

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Defining the model architecture:
</div>

In [ ]:
from keras.applications import Xception

Xception = Xception(weights='imagenet',
                  include_top=True,
                  input_shape=(299, 299, 3))

# remove softmax layer and add 1 neuron for binary classification
fc = keras.layers.Dense(1, activation='sigmoid')(Xception.layers[-2].output)


model = keras.models.Model(inputs=Xception.input, outputs=fc)

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Freezing the weights of the base network:
</div>

In [ ]:
model.trainable = True

for layer in model.layers:
    if layer.name == 'dense_1':
        layer.trainable = True
    else:
        layer.trainable = False

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Looking at the model architecture:
</div>

In [ ]:
model.summary()

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Compiling the model:
</div>

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.RMSprop(lr=1e-5),
              metrics=['accuracy'])

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Training the model:
</div>

In [ ]:
history = model.fit(
      train_dataset,
      epochs=10,
      validation_data=val_dataset)

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Evaluating the model:
</div>

In [ ]:
validation_loss, validation_accuracy = model.evaluate_generator(validation_generator, steps=78)
print('Accuracy on the Validation Set:', validation_accuracy)

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Plotting the result:
</div>

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Finetuning the model by unfreezing some layers:
</div>

In [ ]:
Xception.trainable = True

for layer in Xception.layers:
    if layer.name == 'block14_sepconv1':
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
Xception.summary()

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Compiling the model:
</div>

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.RMSprop(lr=1e-6),
              metrics=['accuracy'])

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Continuing the training:
</div>

In [ ]:
history = model.fit(
      train_dataset,
      epochs=5,
      validation_data=val_dataset)

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Evaluating the model:
</div>

In [ ]:
validation_loss, validation_accuracy = model.evaluate_generator(validation_generator, steps=78)
print('Accuracy on the Validation Set:', validation_accuracy)

<div style="direction:ltr;text-align:left;font-family: Tahoma">
Saving the model:
</div>

In [ ]:
model.save('cnn_transfer_learning.h5')

<div class="alert">
<div style="direction:ltr;text-align:left;font-family:B Tahoma"> Practical Deep Learning Course for Computer Vision
<br>Vahid Reza Khazaie<br>
</div>
<a href="https://www.linkedin.com/in/vahidrezakhazaie/">LinkedIn</a> - <a href="https://github.com/vrkh1996">GitHub</a>

</div>